In [120]:
#essential libraries

#from google_drive import gdrive

from matplotlib.pyplot import show
import numpy as np
import pandas as pd
from scipy.cluster.vq import *
import operator
#import matplotlib
#matplotlib.use('Agg')
from matplotlib import pyplot as plt
import pickle
import shelve
import re
from collections import Counter, defaultdict, OrderedDict, deque
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
import scipy
from sklearn.feature_extraction import DictVectorizer
from matplotlib.backends.backend_pdf import PdfPages
import csv
import sys
import math
from copy import deepcopy, copy
import random
from datetime import datetime, timedelta
from operator import itemgetter
from itertools import chain
import os
import gc
import linecache
import multiprocessing
from functools import partial
from itertools import repeat
from IPython.display import clear_output
import json

import pprint
pp = pprint.PrettyPrinter()

import rdflib
from rdflib.namespace import OWL, RDF, RDFS
from rdflib import URIRef

from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import OneClassSVM, SVC
from sklearn.mixture import GMM
from sklearn.mixture import DPGMM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage, dendrogram
import scipy.cluster.hierarchy as hier
from scipy.cluster.hierarchy import fcluster
from scipy import spatial
from sklearn.multiclass import OneVsRestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics
from scipy.cluster.hierarchy import linkage, dendrogram
import scipy.cluster.hierarchy as hier
from scipy import stats
from numpy.testing import assert_almost_equal

from scipy.spatial.distance import cosine as cosine_similarity

from divergence import gau_js as js_divergence
from brick_parser import tagList, tagsetList, equipTagsetList, pointTagsetList, locationTagsetList,\
equalDict, pointTagList, equipTagList, locationTagList, equipPointDict
subTagListDict = dict([('point', pointTagList),
                          ('equip', equipTagList),
                          ('location', locationTagList)
                         ])
subTagsetListDict = dict([('point', pointTagsetList),
                          ('equip', equipTagsetList),
                          ('location', locationTagsetList)
                         ])

#from cmu_parser import cmu_building_parse

debugFlag = False

In [121]:
import imp
import building_tokenizer
imp.reload(building_tokenizer)
import building_tokenizer as toker
from functools import reduce

building_name = 'ap_m'
#building_name = 'ebu3b'
notUcsdBuildings = ['ghc']

tokenType = 'NoNumber'
if not building_name in notUcsdBuildings:
    naeDict = dict()
    naeDict['bonner'] = ["607", "608", "609", "557", "610"]
    naeDict['ap_m'] = ['514', '513','604']
    naeDict['bsb'] = ['519', '568', '567', '566', '564', '565']
    naeDict['ebu3b'] = ["505", "506"]
    #naeDict['otterson'] = ["518", "517", "589", "590"]
    naeList = naeDict[building_name]

    labeledFile = 'metadata/' + building_name + '_sensor_types_location.csv'
    with open(labeledFile, 'rb') as fp:
        #truthDF = pd.read_excel(fp)
        truthDF = pd.DataFrame.from_csv(fp)
        #truthDF = truthDF.set_index(keys='Unique Identifier')

    wordFeatFile = 'data/wordfeat_'+building_name+'.pkl'

    tokenTypeList = ['NoNumber', 'Alphanumeric', 'AlphaAndNum', 'NumAsSingleWord']

    bacnetTypeMapDF = pd.DataFrame.from_csv('metadata/bacnettype_mapping.csv')
    unitMap = pd.read_csv('metadata/unit_mapping.csv').set_index('unit')
    for val in Counter(unitMap.keys()).values():
        if val>1:
            "Unit map file ERROR"
            assert(False)
            
            
    trueDF = pd.DataFrame.from_csv('metadata/'+building_name+'_sensor_types_location.csv')
    sensorDF, nameList, jcinameList, descList, unitList, bacnettypeList, wordList = \
    toker.structure_metadata(buildingName=building_name, tokenType=tokenType, \
                             validSrcidList=trueDF.index.tolist(), withDotFlag=False)

    origSensorDF = deepcopy(sensorDF)
    origNameList = deepcopy(nameList)
    origJcinameList = deepcopy(jcinameList)
    origDescList = deepcopy(descList)
    origUnitList = deepcopy(unitList)
    origBacnettypeList = deepcopy(bacnettypeList)
    origWordList = deepcopy(wordList)

    #_, rawNameList, rawJcinameList, rawDescList, _, _, _ = toker.structure_metadata(building_name=building_name, tokenType='AlphaAndNum', \
    _, rawNameList, rawJcinameList, rawDescList, _, _, _ =\
    toker.structure_metadata(buildingName=building_name, tokenType='JustSeparate', \
                             validSrcidList=trueDF.index.tolist(), withDotFlag=False)
    
    
else: 
    filename = 'metadata/'+building_name+'_sensor_types_location.csv'
    #   filename = 'metadata/%s_sensor_types_location.csv'%building_name
    df = pd.read_csv(filename)
    sentenceDict = dict()    
    for i,raw in enumerate(df['bas_raw'].tolist()):
        sentenceDict[i] = toker.tokenize(tokenType, raw)
    
    
## Common part
    
#with open('data/'+building_name+'_str_score_dict.pkl', 'rb') as fp:
#    scoreDictDict = pickle.load(fp)
#    ### If a word is exactly matched with another, fix it.
#    for word, scoreDict in scoreDictDict.items():
#        if word in scoreDict.keys():
#            scoreDictDict[word] = {word:1}
#    scoreDictDict['co'] = {'co2':1}
        
adder = lambda x,y:x+y
splitter = lambda x:x.split()
bacnetTypes = list(set(reduce(adder,map(splitter,origBacnettypeList),[])))
#for bacnetType in bacnetTypes:
#    scoreDictDict[bacnetType] = {bacnetType:1}
#units = list(set(unitMap['word'].tolist()))
#for unit in units:
#    if type(unit)==str:
#        scoreDictDict[unit] = {unit:1}

In [122]:
with open("metadata/%s_sentence_dict_justseparate.json"%building_name, "r") as fp:
    sentenceDict = json.load(fp)

In [123]:
known_label_dict = dict()

known_label_dict['ebu3b'] = {
    'nae':'network_adapter',
    'n': 'network_adapter',
    'ebu': 'building',
    'b': 'identifier', # b could be basement, which is ignored currently.
    'rm':'room',
    'vma':'vav',
    '\n':'none',
    'vnd':'vnd',
    'cmd':'command',
    'zn':'zone',
    'chilled_water_pump': 'chwp',
    'clgpid':'cooling_command',
    'htgpid':'heating_command',
    'rh':'reheat',
    'vlv':'valve',
    'supflo':'supply_air_flow',
    'sup':'supply',
    'rf':'return_fan',
    'clgmaxflo':'cooling_max_supply_air_flow_setpoint',
    'dx':'network_adapter',
    'hwp':'hot_water_pump',
    'sys':'system',
    'dpr':'damper',
    'chwp':'chilled_water_pump',
    'stpt':'setpoint',
    'clgminflo':'cooling_min_supply_air_flow_setpoint',
    'clg':'cooling',
    'dx':'network_adapter',
    'dmpr':'damper',
    'pos':'position',
    'actclgsp':'effective_cooling_temperature_setpoint',
    'clgminflo':'cooling_min_supply_air_flow_setpoint',
    'dpr':'damper',
    'clgmaxflo':'cooling_max_supply_air_flow_setpoint',
    'adj':'adjust',
    'commonsp':'temperature_setpoint',
    'htgflo':'heating_supply_air_flow_setpoint',
    'htg':'heating',
    'acthtgsp':'effective_heating_temperature_setpoint',
    'sen':'sensor',
    'supflosp':'supply_air_flow_setpoint',
    'dmprpos':'damper_position',
    'htgflow':'heating_supply_air_flow_setpoint',
    'sf':'supply_fan',
    'chw': 'chilled_water',
    'alm':'alarm',
    'common':'none',
    'ai':'unknown',
    'ao':'unknown',
    'do':'unknown',
    'o': 'none',
    'programming': 'nope',
    'pid':'none',
    'ah':'ahu',
    'w':'warm',
    'a':'identifier',
    'freq':'frequency',
    'rate': 'rated'
    
}

known_label_dict['ap_m'] = {
    'ef': 'exhaust_fan',
    'stg': 'stage',
    'nae':'network_adapter',
    'n': 'network_adapter',
    'htg': 'heating',
    'ap': 'building',
    'm': 'building',
    'rm': 'room',
    'vlv': 'valve',
    'zn': 'zone',
    'trunk': 'network_adapter',
    'dx': 'network_adapter',
    'aclg': 'cooling',
    'stpt': 'setpoint',
    'programming': 'none',
    'actual': 'none',
    'wc': 'warm_cool',
    'adj': 'adjust',
    'ohmin': 'occupied_heating_min',
    'minimum': 'min',
    'cmd': 'command',
    'oclg': 'occupied_cooling',
    'supflo':'supply_flow',
    'co':'co2',
    'flows': 'flow',
    'ocmin': 'occupied_cooling_min',
    'ohmin': 'occupied_heating_min',
    'adjustment': 'adjust',
    'dmpr': 'damper',
    'position': 'position',
    'sup': 'supply',
    'hmax': 'heating_max',
    'maximum': 'max',
    'ena': 'enable',
    'chw': 'chilled_water',
    'chwp': 'chilled_water_pump',
    'fwdrev': 'direction',
    'sts': 'status',
    'clg':'cooling',
    'r': 'right_identifier',
    'flw':'flow',
    'cmax':'cooling_max',
    'ahtg':'heating',
    'hhw': 'unknown',
    'alm': 'alarm',
    'tuning': 'none',
    'parameters':'none',
    'rcdb': 'unknown',
    'chwdp': 'chilled_water_differential_pressure',
    'cmaxflow': 'cooling_max_flow',
    'pos': 'position',
    'volts': 'voltage',
    'rhc': 'unknown',
    'ohtg': 'occupied_heating',
    'drv': 'drive',
    'da': 'discharge_air',
    'satstpt': 'supply_air_temperature_setpoint',
    'sathtg': 'supply_air_temperature_heating',
    'pb': 'proportional_band',
    'ah':'ahu',
    'curr': 'current',
    'prop': 'proportional',
    'occ': 'occupied',
    'unocc': 'unoccupied',
    'bsmtw': 'name',
    'hall': 'name',
    'lmt': 'limit',
    'common': 'temperature',
    'occhtgfl': 'occupied_heating_flow',
    'easp': 'exhaust_air_static_pressure',
    'feedback': 'sensor',
    'vp': 'velocity_pressure',
    'esp':'exhaust_air_static_pressure',
    'fle': 'name',
    'hx': 'heat_exchanger',
    'pmp': 'pump',
    'freq': 'frequency',
    'cmaxflo':'cooling_max_flow',
    'enble': 'enable',
    'rst':'reset',
    'pan': 'panel',
    'lowosa': 'low_outside_air',
    'diff': 'differential',
    'osa': 'outside_air',
    'dpr': 'damper',
    'sasp': 'supply_air_static_pressure',
    'it': 'integral',
    'forward': 'direction',
    'backward': 'direction',
    'lvl': 'level',
    'sat': 'supply_air_temperature',
    'sf': 'supply_fan',
    'int': 'integral',
    'omin': 'occupied_min',
    'bsmt': 'basement',
    'flt': 'fault',
    'unt': 'unknown',
    'ra': 'return_air',
    'avg': 'average',
    'altclgdb': 'altclgdb',
    'optimization': 'none',
    
}

known_word_list = known_label_dict[building_name].keys()

In [124]:
def validate_label(label):
    nonBrickTagList = ['none', 'right_identifier', 'left_identifier',
                       'network_adapter', 'vnd', 'unknown', 'name']
    # name: stand alone unique name.
    # left identifier: contraints meaning of left tagset
    # right identifier: contraints meaning of right tagset
    if label in nonBrickTagList:
        return True
    token_list = label.split('_')
    valid_label_list = tagList + tagsetList
    for token in token_list:
        valid_label_list = [label for label in valid_label_list if token in label]
    if len(valid_label_list)>0:
        return True
    else:
        return False
        

def ask_expert_labels(sentence):
    # Input: List of words
    # Output: feature list:
    # Human operation: Give labels on the words.
    sliceSize = 2
    featureDict = dict()

    #for words in rolling_window(sentence, 2):
    #    for word in wo
    print("\nGiven sentence: %s"%sentence)
    labelList = list()
    for word in sentence:
        if re.match('[^0-9a-zA-Z]+', word):
            labelList.append('none')
            continue
        if re.match('\d+', word):
            labelList.append('left_identifier')
            continue
        if word in known_word_list:
            labelList.append(known_label_dict[building_name][word])
            continue
        if word in tagList:
            labelList.append(word)
            continue
            
        while True:
            label = input(str(word)+': \n')
            if validate_label(label):
                labelList.append(label)
                break
            else:
                print("Pick Tag among the following tags: " + str(tagList))
    return labelList

In [125]:
labelFilename = 'metadata/%s_label_dict_justseparate.json'%building_name
if os.path.isfile(labelFilename):
    with open(labelFilename, 'r') as fp:
        labelListDict = json.load(fp)
else:
    labelListDict = {}
with open(labelFilename+'.backup', 'w') as fp:
    json.dump(labelListDict, fp, indent=2)

In [126]:
labelSampleNum = 50
for i in random.sample(range(0,len(sensorDF)), labelSampleNum):
    srcid = sensorDF.iloc[i].name
    if not srcid in labelListDict.keys():
        print(srcid)
        labelListDict[srcid] = ask_expert_labels(sentenceDict[srcid])
        

514_1_3006869

Given sentence: ['nae', '14', 'n', '2', 'trunk', '2', 'vav', '32', 'supflo', 'stpt', '\n', 'ap', '&', 'm', '.', 'rm', '-', '5816', '.', 'supflo', '-', 'stpt', '\n', 'supply', 'air', 'flow', 'setpoint', '\n']
513_0_3005476

Given sentence: ['nae', '13', 'n', '2', 'trunk', '1', 'vfd', '6', 'drv', 'tmp', '\n', 'ap', '&', 'm', '.', 'ahu', '3', '-', 'l', 'vfd', '.', 'drv', '-', 'tmp', '\n', 'vfd', 'temperature', '\n']
tmp: 
temperature
l: 
right_identifier
tmp: 
temperature
604_0_3012416

Given sentence: ['nae', '104', 'n', '2', 'trunk', '1', 'vav', '37', 'zn', 't', '\n', 'ap', '&', 'm', '.', 'rm', '-', '6161', '.', 'zn', '-', 't', '\n', 'zone', 'temperature', '\n']
t: 
temperature
t: 
temperature
514_1_3006512

Given sentence: ['nae', '14', 'n', '2', 'trunk', '2', 'vav', '4', 'ohtg', 'stpt', '\n', 'ap', '&', 'm', '.', 'rm', '-', 'b', '859', '.', 'ohtg', '-', 'stpt', '\n', 'occupied', 'heating', 'setpoint', '\n']
b: 
left_identifier
513_1_3004825

Given sentence: ['nae', '13'

In [130]:
print(len(labelListDict))
with open(labelFilename, 'w') as fp:
    json.dump(labelListDict,fp, indent=4, separators=(',', ': '))

455


In [128]:
wrongSrcidList = list()
for srcid, sentence in sentenceDict.items():
    if 't' in sentence and '5' in sentence:
        #wrongSrcidList.append(srcid)
        if srcid in labelListDict.keys():
            print(srcid, sentence)

513_0_3004810 ['nae', '13', 'n', '2', 'trunk', '1', 'vav', '5', 'zn', 't', '\n', 'ap', '&', 'm', '.', 'rm', '-', 'b', '349', '.', 'zn', '-', 't', '\n', 'zone', 'temperature', '\n']
513_0_3005621 ['nae', '13', 'n', '2', 'trunk', '1', 'vav', 'b', '5', 'zn', 't', '\n', 'ap', '&', 'm', '.', 'rm', '-', 'b', '412', '.', 'zn', '-', 't', '\n', 'zone', 'temperature', '\n']


In [129]:
sensorDF.iloc[0]

description        HX2 Medium Temp Water Supply
jci_name                     AP&M.HX-2.HX2-MTWS
name           NAE 14 N2 Trunk 2 AHU 6 HX2 MTWS
type                                          0
type_string                        Analog Input
unit                                         64
Name: 514_0_3006333, dtype: object